In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_parquet("../data/train.parquet")
train_df

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
216716091,12899776,1737908,1661723987,0
216716092,12899777,384045,1661723976,0
216716093,12899777,384045,1661723986,0
216716094,12899778,561560,1661723983,0


In [2]:
seven_days = 7*24*60*60
train_cutoff = train_df.ts.max() - seven_days
train_df = train_df[train_df.ts <= train_cutoff]
seven_days = 7*24*60*60
train_cutoff = train_df.ts.max() - seven_days
train_df = train_df[train_df.ts <= train_cutoff]

In [3]:
train_df.session.max()

8643219

In [4]:
clicks_count = train_df[train_df['type'] == 0].groupby("aid")[['aid']].agg("size")
carts_count = train_df[train_df['type'] == 1].groupby("aid")[['aid']].agg("size")
orders_count = train_df[train_df['type'] == 2].groupby("aid")[['aid']].agg("size")
train_df['item_click_count'] = train_df['aid'].map(clicks_count)
train_df['item_carts_count'] = train_df['aid'].map(carts_count)
train_df['item_orders_count'] = train_df['aid'].map(orders_count)
train_df = train_df.fillna(0)

In [5]:
train_df

,session,aid,ts,type,item_click_count,item_carts_count,item_orders_count
0,0,1517085,1659304800,0,65.0,11.0,3.0
1,0,1563459,1659304904,0,48.0,0.0,0.0
2,0,1309446,1659367439,0,2691.0,339.0,52.0
3,0,16246,1659367719,0,640.0,65.0,18.0
4,0,1781822,1659367871,0,26.0,1.0,0.0
...,...,...,...,...,...,...,...
175864800,8643215,1463083,1660514398,0,31.0,3.0,0.0
175864804,8643216,1727218,1660514398,0,2049.0,77.0,9.0
175864821,8643217,595370,1660514399,0,445.0,12.0,0.0
175864823,8643218,585048,1660514399,0,3229.0,23.0,2.0


In [6]:
item_feat = train_df.drop_duplicates("aid")[['aid','item_click_count','item_carts_count','item_orders_count']]
item_feat

,aid,item_click_count,item_carts_count,item_orders_count
0,1517085,65.0,11.0,3.0
1,1563459,48.0,0.0,0.0
2,1309446,2691.0,339.0,52.0
3,16246,640.0,65.0,18.0
4,1781822,26.0,1.0,0.0
...,...,...,...,...
175854509,802266,2.0,0.0,0.0
175858780,1494543,1.0,0.0,0.0
175864220,646053,1.0,1.0,0.0
175864322,1609192,1.0,0.0,0.0


In [7]:
item_feat.to_parquet("../data/feat/item_feat.parquet")